# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('jurisprudencia_selecionada_enunciados.txt', sep = '|')
df.head()

,COD,NUM_ENUNCIADO,COD_AREA,DESCR_AREA,COD_TEMA,DESCR_TEMA,COD_SUBTEMA,DESCR_SUBTEMA,COD_DOC_TRAMITAVEL_ENUNCIADO,TEXTO_ENUNCIADO,ACORDAO,TIPO_PROCESSO
0,1400,1236,50,Responsabilidade,488,Solidariedade,261,Benefício previdenciário,54995437,Não comprovada a participação do beneficiário ...,Acórdão 297/2016 - PL,Tomada de Contas Especial
1,1700,1534,46,Finanças Públicas,981,Exportação,983,Petróleo,55025587,A operação ficta de exportação de plataformas ...,Acórdão 366/2016 - PL,Solicitação do Congresso Nacional
2,5700,5314,50,Responsabilidade,203,Multa,1021,Dosimetria,55455370,"No âmbito do TCU, a dosimetria da pena tem com...",Acórdão 944/2016 - PL,Acompanhamento
3,284,40,45,Direito Processual,162,Princípio da independência das instâncias,481,Decisão judicial,54773746,O princípio da independência das instâncias pe...,Acórdão 30/2016 - PL,Tomada de Contas Especial
4,298,54,49,Pessoal,141,Sistema S,142,Nepotismo,54773402,É vedado aos dirigentes das entidades do Siste...,Acórdão 55/2016 - PL,Representação


In [2]:
df.shape

(13312, 12)

In [3]:
df.groupby(['DESCR_AREA']).size()

DESCR_AREA
Competência do TCU          557
Contrato Administrativo     942
Convênio                    685
Desestatização              140
Direito Processual         1813
Finanças Públicas           328
Gestão Administrativa       339
Licitação                  2765
Pessoal                    3396
Responsabilidade           2347
dtype: int64

In [4]:
areas = df.groupby(['DESCR_AREA']).groups.keys()
areas

dict_keys(['Competência do TCU', 'Contrato Administrativo', 'Convênio', 'Desestatização', 'Direito Processual', 'Finanças Públicas', 'Gestão Administrativa', 'Licitação', 'Pessoal', 'Responsabilidade'])

In [5]:
from sklearn.preprocessing import LabelBinarizer

lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [6]:
y = lbArea.transform(df['DESCR_AREA'])
y.shape

(13312, 10)

In [9]:
from keras.preprocessing.text import Tokenizer
import numpy as np

limite_texto = 200
dim_vetor = 100

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['TEXTO_ENUNCIADO'])

word_index = tokenizer.word_index
vocabulario = len(word_index) + 1
print('Found %s unique tokens.' % len(word_index))

sequences = tokenizer.texts_to_sequences(df['TEXTO_ENUNCIADO'])

Found 15387 unique tokens.


In [10]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (13312, 200)


In [33]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('model.txt')

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [35]:
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocabulario, 100))

ok = 0
for word, i in tokenizer.word_index.items():
    if word in model:
        embedding_matrix[i] = model[word]
        ok += 1
print('Vocabulario:', i)
print('Encontrados no modelo:', ok, '=', ok * 100. / i)

Vocabulario: 15387
Encontrados no modelo: 13758 = 89.41314096315071


# Treinamento

In [36]:
from keras.layers import Embedding

embedding = Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False)

In [37]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
from keras.layers.core import Dropout

model = Sequential()
model.add(embedding)
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
flatten_4 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 2048)              40962048  
_________________________________________________________________
dropout_7 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dropout_8 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)                10250     
Total para

In [38]:
history = model.fit(x, y, epochs=50, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10649 samples, validate on 2663 samples
Epoch 1/50
10649/10649 [==============================] - 18s 2ms/step - loss: 1.8993 - categorical_accuracy: 0.3763 - val_loss: 1.7048 - val_categorical_accuracy: 0.3522
Epoch 2/50
10649/10649 [==============================] - 17s 2ms/step - loss: 1.3523 - categorical_accuracy: 0.5499 - val_loss: 1.2747 - val_categorical_accuracy: 0.5854
Epoch 3/50
10649/10649 [==============================] - 17s 2ms/step - loss: 1.0981 - categorical_accuracy: 0.6315 - val_loss: 1.5028 - val_categorical_accuracy: 0.5216
Epoch 4/50
10649/10649 [==============================] - 17s 2ms/step - loss: 0.8864 - categorical_accuracy: 0.7065 - val_loss: 1.3539 - val_categorical_accuracy: 0.5734
Epoch 5/50
10649/10649 [==============================] - 17s 2ms/step - loss: 0.7200 - categorical_accuracy: 0.7626 - val_loss: 1.4315 - val_categorical_accuracy: 0.5674
Epoch 6/50
10649/10649 [==============================] - 17s 2ms/step - loss: 0.6026 - categori

10649/10649 [==============================] - 16s 2ms/step - loss: 0.0429 - categorical_accuracy: 0.9879 - val_loss: 3.1950 - val_categorical_accuracy: 0.6128
Epoch 49/50
10649/10649 [==============================] - 16s 2ms/step - loss: 0.0477 - categorical_accuracy: 0.9860 - val_loss: 3.5014 - val_categorical_accuracy: 0.6046
Epoch 50/50
10649/10649 [==============================] - 16s 2ms/step - loss: 0.0634 - categorical_accuracy: 0.9835 - val_loss: 3.6145 - val_categorical_accuracy: 0.6012


In [40]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 18s 2ms/step - loss: 1.7227 - categorical_accuracy: 0.4257 - val_loss: 1.8049 - val_categorical_accuracy: 0.3718
Epoch 2/20
10649/10649 [==============================] - 16s 2ms/step - loss: 1.0776 - categorical_accuracy: 0.6377 - val_loss: 1.6382 - val_categorical_accuracy: 0.5017
Epoch 3/20
10649/10649 [==============================] - 16s 2ms/step - loss: 0.7322 - categorical_accuracy: 0.7580 - val_loss: 1.3846 - val_categorical_accuracy: 0.5667
Epoch 4/20
10649/10649 [==============================] - 16s 2ms/step - loss: 0.4503 - categorical_accuracy: 0.8495 - val_loss: 1.5817 - val_categorical_accuracy: 0.5633
Epoch 5/20
10649/10649 [==============================] - 16s 2ms/step - loss: 0.2684 - categorical_accuracy: 0.9105 - val_loss: 1.6906 - val_categorical_accuracy: 0.6226
Epoch 6/20
10649/10649 [==============================] - 16s 2ms/step - loss: 0.1774 - categori

In [41]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 17s 2ms/step - loss: 1.7546 - categorical_accuracy: 0.4069 - val_loss: 1.4525 - val_categorical_accuracy: 0.5257
Epoch 2/20
10649/10649 [==============================] - 16s 1ms/step - loss: 1.1966 - categorical_accuracy: 0.5977 - val_loss: 1.6804 - val_categorical_accuracy: 0.4412
Epoch 3/20
10649/10649 [==============================] - 16s 2ms/step - loss: 0.8706 - categorical_accuracy: 0.7029 - val_loss: 1.3191 - val_categorical_accuracy: 0.5719
Epoch 4/20
10649/10649 [==============================] - 16s 2ms/step - loss: 0.6072 - categorical_accuracy: 0.7910 - val_loss: 1.4491 - val_categorical_accuracy: 0.5599
Epoch 5/20
10649/10649 [==============================] - 16s 1ms/step - loss: 0.3982 - categorical_accuracy: 0.8611 - val_loss: 1.6902 - val_categorical_accuracy: 0.5513
Epoch 6/20
10649/10649 [==============================] - 16s 1ms/step - loss: 0.2412 - categori